In [1]:
!cat ../README.md

# Super Resolution, On The Edge, For The Win!
*by Akshay Chawla and Priyam Tejaswin*

Main reference paper : [Real-Time Video Super-Resolution with Spatio-Temporal Networks and Motion Compensation, CVPR 2017](http://openaccess.thecvf.com/content_cvpr_2017/papers/Caballero_Real-Time_Video_Super-Resolution_CVPR_2017_paper.pdf)
```
.
├── README.md
├── papers ## PDFs for review and reference (not committed).
├── src_tensorflow ## Code in tf (using `tensorflow.keras`). 
└── src_tf2 ## Code in tf2_alpha.
```

## Dependencies
- `src_tensorflow` uses **tensorflow 1.13.1**. This is the latest stable release as of creating this repository.
- `src_tf2` uses **tensorflow 2.0** which is still an alpha build.

## System Design
![](system_blocks.jpg)

## References
- VESPCN implementation (unofficial, TF) - <https://github.com/LoSealL/VideoSuperResolution/blob/master/VSR/Models/Vespcn.py>
- Fully-featured TF2 implementation of YOLOv3 (for checking TF2 oddities) - <https://github.com/zzh8829/yolov3-tf

In [2]:
import tensorflow as tf

In [3]:
tf.enable_eager_execution()
print tf.executing_eagerly()

True


In [136]:
dataset = tf.data.TextLineDataset("../README.md")
dataset = dataset.map(lambda line: tf.string_split([line]).values)
dataset = dataset.filter(lambda row: tf.size(row) > 3)
dataset = dataset.map(lambda row: tf.slice(row, [0], [2]))
dataset = dataset.window(size=3, shift=1, drop_remainder=True) ## THIS WILL RETURN A DATASET!!!!
dataset = dataset.flat_map(lambda dset: dset.batch(3))
# How to shuffle?

In [135]:
for i in range(5):
    for batch in dataset:
        print i, batch

tf.Tensor(# Super Resolution, On The Edge, For The Win!, shape=(), dtype=string)
tf.Tensor(*by Akshay Chawla and Priyam Tejaswin*, shape=(), dtype=string)
tf.Tensor(, shape=(), dtype=string)
tf.Tensor(Main reference paper : [Real-Time Video Super-Resolution with Spatio-Temporal Networks and Motion Compensation, CVPR 2017](http://openaccess.thecvf.com/content_cvpr_2017/papers/Caballero_Real-Time_Video_Super-Resolution_CVPR_2017_paper.pdf), shape=(), dtype=string)
tf.Tensor(```, shape=(), dtype=string)
tf.Tensor(., shape=(), dtype=string)
tf.Tensor(├── README.md, shape=(), dtype=string)
tf.Tensor(├── papers ## PDFs for review and reference (not committed)., shape=(), dtype=string)
tf.Tensor(├── src_tensorflow ## Code in tf (using `tensorflow.keras`). , shape=(), dtype=string)
tf.Tensor(└── src_tf2 ## Code in tf2_alpha., shape=(), dtype=string)
tf.Tensor(```, shape=(), dtype=string)
tf.Tensor(, shape=(), dtype=string)
tf.Tensor(## Dependencies, shape=(), dtype=string)
tf.Tensor(- `src_ten

In [91]:
!pwd

/Users/tejaswin.p/THIS_LAPTOP_projects/srote-ftw/src_tensorflow


In [93]:
!ls

dump_frames.py rough.ipynb


In [95]:
os.path.isdir('helloworld')

False

In [96]:
os.listdir('.')

['rough.ipynb', 'dump_frames.py', '.ipynb_checkpoints']

## Loading frames.

In [47]:
import os
import itertools

VFDEL = '___' ## VideoName___FrameId delimiter.
NFRAMES = 2 ## Number of frames considered per sample.

def _fskey(f):
    vname, fid = f.rstrip('.png').split(VFDEL)
    fid = int(fid)
    return vname, fid

def build_dataset(fdir):
    all_frames = []

    vdirs = [os.path.join(fdir, d) for d in os.listdir(fdir)]
    vdirs = [d for d in vdirs if os.path.isdir(d)]
    print "Found %d vid files."%len(vdirs)

    for dpath in vdirs:
        print "In %s"%dpath
        frames = [f for f in os.listdir(dpath) if f.endswith('.png')]
        print "\tFound %d frames."%len(frames)

        frames = sorted(frames, key=lambda x:_fskey(x))
        frames = [os.path.join(dpath, f) for f in frames]

        ## Group sorted frames by NFRAMES.
        motion_frames = []
        for i in range(len(frames) - NFRAMES + 1):
            motion_frames.append(tuple(frames[i : i+NFRAMES]))

        ## Add to master list.
        all_frames.extend(motion_frames)

    print "Found %d frame groups of length %d."%(len(all_frames), NFRAMES)
    return all_frames

In [48]:
batched_fnames = build_dataset('../data/frames/')

Found 2 vid files.
In ../data/frames/hall_monitor_cif.y4m
	Found 300 frames.
In ../data/frames/sample_hd_video.mp4
	Found 376 frames.
Found 674 frame groups of length 2.


In [81]:
## Using generator...
# dataset = tf.data.Dataset.from_generator(lambda: batched_fnames[:10], (tf.string, tf.string))
# dataset = dataset.shuffle(buffer_size=len(batched_fnames))
# dataset = dataset.batch(3)

## Using tensor_slices...
dataset = tf.data.Dataset.from_tensor_slices(batched_fnames[:3])
# dataset = dataset.shuffle(buffer_size=len(batched_fnames))
dataset = dataset.flat_map(tf.data.Dataset.from_tensor_slices) ## Order will be preserved | https://stackoverflow.com/questions/51015918
dataset = dataset.map(load_image)
# dataset = dataset.map(process_image)

for row in dataset:
    print row.shape

(1, 288, 352, 3)
(1, 288, 352, 3)
(1, 288, 352, 3)
(1, 288, 352, 3)
(1, 288, 352, 3)
(1, 288, 352, 3)


In [152]:
import matplotlib.pyplot as plt
%matplotlib inline

In [150]:
img = np.arange(36 * 2).reshape([1, 6, 6, 2])
print img[0, :, :, 0]
print img[0, :, :, 1]

[[ 0  2  4  6  8 10]
 [12 14 16 18 20 22]
 [24 26 28 30 32 34]
 [36 38 40 42 44 46]
 [48 50 52 54 56 58]
 [60 62 64 66 68 70]]
[[ 1  3  5  7  9 11]
 [13 15 17 19 21 23]
 [25 27 29 31 33 35]
 [37 39 41 43 45 47]
 [49 51 53 55 57 59]
 [61 63 65 67 69 71]]


In [155]:
patches = tf.image.extract_image_patches(img, ksizes=[1, 4, 4, 1], strides=[1, 2, 2, 1], rates=[1, 1, 1, 1], padding='VALID')

patches.numpy().shape
print patches[0, 0, 0].numpy().reshape([4, 4, 2])[:, :, 0]
print patches[0, 0, 0].numpy().reshape([4, 4, 2])[:, :, 1]

[[ 0  2  4  6]
 [12 14 16 18]
 [24 26 28 30]
 [36 38 40 42]]
[[ 1  3  5  7]
 [13 15 17 19]
 [25 27 29 31]
 [37 39 41 43]]


In [78]:
def load_image(fpath):
    """
    Load and 'snap' to compatible size (extract 96x96 grids).
    """
    image_string = tf.read_file(fpath)
    image = tf.image.decode_png(image_string, channels=3)
    image = tf.expand_dims(image, 0)
    return image

In [79]:
def process_image(image):
    patches = tf.image.extract_image_patches(
        image,
        ksizes = [1, 96, 96, 1],
        strides = [1, 14, 14, 1],
        rates = [1, 1, 1, 1],
        padding = 'VALID'
    )
    return patches

In [71]:
a = range(25)
print a

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]


In [ ]:
i = 0
while i < len(a):
    

In [ ]:
25 - 